## Config .env
DB_USER=user_securite_routiere
DB_PASSWORD=password123
DB_HOST=localhost
DB_PORT=5432
DB_NAME=securite_routiere_db

Une fois que le fichier .env est bien configuré on peut lancer ça et commencer à travailler

In [ ]:
import sys
sys.path.append("..")  # remonte d'un niveau vers le dossier parent
from utils.utils import connect_to_db, insert_data_to_db, mapping_colonnes, fetch_data_from_db
engine = connect_to_db()
df = fetch_data_from_db(table_name='accidents_raw', engine=engine, schema='bronze')
df

In [ ]:
df_accident = df.copy()

Dès qu'on a le dataframe prêt à être insérer dans la table on peut utiliser insert_data_to_db :

In [ ]:
insert_data_to_db(df_accident, table_name='accidents', engine=engine, schema='silver') # Par exemple pour insérer dans la table 'accidents' du schéma 'silver'